In [3]:
import os
import shutil
import json
import time
import re
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

import datasets
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import evaluate
import numpy as np

UKRAINIAN_LETTERS = 'абвгґдеєжзиіїйклмнопрстуфхцчшщьюя'
UKRAINAIN_VOWELS = 'аеєиіїоуюя'
ENGLISH_LETTERS = 'abcdefghijklmnopqrstuvwxyz'

In [4]:
allowed_punctuation = """ .,!?;:'"«»()+-—–"""
other_punctuation =  """ $%&<>{}[]*"""

voa_df = pd.read_csv('./voa_stressed_cleaned_data.csv')
unique_letters = set(''.join(voa_df['text'].to_list()))

unique_letters = unique_letters - set(UKRAINIAN_LETTERS) \
                                - set(UKRAINIAN_LETTERS.upper()) \
                                - set(allowed_punctuation) \
                                - set(other_punctuation)

df = voa_df[~voa_df['text'].apply(lambda x: any(c in unique_letters for c in x))]
df = df[['text', 'labels']]
df = df.rename(columns={
    'labels': 'label'
})
df.shape

(137078, 2)

In [10]:
from datasets import load_from_disk
tokenized_dataset = load_from_disk("tokenized_voa_dataset")

model_id = 'google/byt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
model_pretrained = '/byt5-small-accentor-model/checkpoint-49000/'
model = AutoModelForSeq2SeqLM.from_pretrained(model_pretrained)

# Evaluate custom text

In [ ]:
def generate_prediction(input_text, model, tokenizer, device='cuda'):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1500)

    decoded_preds = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded_preds
    
input_text = "Привіт, як справи?"
decoded_prediction = generate_prediction(input_text, model, tokenizer)
print(decoded_prediction)

input_text = "відкрити ключем замок"
decoded_prediction = generate_prediction(input_text, model, tokenizer)
print(decoded_prediction)

input_text = "Сидить бобер на березі"
decoded_prediction = generate_prediction(input_text, model, tokenizer)
print(decoded_prediction)

input_text = "Листя на березі"
decoded_prediction = generate_prediction(input_text, model, tokenizer)
print(decoded_prediction)

# Get metrics

In [22]:
import re
import numpy as np

UKRAINIAN_VOWELS = 'аеєиіїоуюя'

def compute_stress_metrics_from_df(df_predictions):
    def split_with_stress(sentence):
        return re.split(r'(\s+|-)', sentence)

    match_results = []
    match_sentence_results = []
    for _, row in df_predictions.iterrows():
        pred = row["Predictions"]
        label = row["True Labels"]

        if pred == label:
            match_sentence_results.append(1)
        else:
            match_sentence_results.append(0)

        pred_split = split_with_stress(pred)
        label_split = split_with_stress(label)

        if len(pred_split) == len(label_split):
            matches = []
            for p, l in zip(pred_split, label_split):
                if not any(char in UKRAINIAN_VOWELS for char in l):
                    continue
                if p == l:
                    matches.append(1)
                elif p.replace('+', '') != l.replace('+', ''):
                    matches = [0]
                    break
                else:
                    matches.append(0)
            if not matches:
                matches = [0]
            match_results.append(np.mean(matches))
        else:
            match_results.append(0)

    return {"average_word_accuracy": np.mean(match_results) * 100, "average_sentence_accuracy": np.mean(match_sentence_results) * 100,}

In [16]:
from IPython.display import display, HTML
model.eval()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

all_predictions = []
i = 0

with torch.no_grad():
    for batch in tqdm(tokenized_dataset["eval"]):
        # i += 1
        # if i==10: break

        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).unsqueeze(0).to(device)
        
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1500)
        decoded_preds = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        labels = batch["label"]
        labels = [l for l in labels if l != -100]
        decoded_labels = tokenizer.decode(labels, skip_special_tokens=True)
        decoded_inputs = tokenizer.decode(batch["input_ids"], skip_special_tokens=True)
        
        all_predictions.append((decoded_inputs, decoded_preds, decoded_labels))

df_predictions = pd.DataFrame(all_predictions, columns=["Input Text", "Predictions", "True Labels"])    

100%|██████████| 1371/1371 [22:49<00:00,  1.00it/s] 


In [23]:
compute_stress_metrics_from_df(df_predictions)

{'average_word_accuracy': 96.15945256471453,
 'average_sentence_accuracy': 73.8147337709701}

# Show errors

In [19]:
def highlight_differences(prediction, label):
    pred_words = re.split(r'(\s+|-)', prediction)
    label_words = re.split(r'(\s+|-)', label)

    highlighted_pred = []
    highlighted_label = []

    for pred_word, label_word in zip(pred_words, label_words):
        if pred_word != label_word:
            highlighted_label.append(f'<span style="color: red; font-weight: bold;">{label_word}</span>')
            highlighted_pred.append(f'<span style="color: red; font-weight: bold;">{pred_word}</span>')
        else:
            highlighted_label.append(label_word)
            highlighted_pred.append(pred_word)

    # Join the lists back into strings
    highlighted_pred = ''.join(highlighted_pred)
    highlighted_label = ''.join(highlighted_label)
    return highlighted_pred, highlighted_label

highlighted_preds = []
highlighted_labels = []
mismatch_mask = []

df_pred_highlighted = df_predictions.copy()

for pred, label in zip(df_pred_highlighted["Predictions"], df_pred_highlighted["True Labels"]):
    highlighted_pred, highlighted_label = highlight_differences(pred, label)
    highlighted_preds.append(highlighted_pred)
    highlighted_labels.append(highlighted_label)
    mismatch_mask.append(pred != label)

df_pred_highlighted["Predictions"] = highlighted_preds
df_pred_highlighted["True Labels"] = highlighted_labels

## Examples of inference on eval set (all)

In [20]:
# all results
display(HTML(df_pred_highlighted.head(20).to_html(escape=False)))

,Input Text,Predictions,True Labels
0,Тридцять років тому.,Тр+идцять р+оків т+ому.,Тр+идцять р+оків т+ому.
1,Потім було музичне училище по класу ударних інструментів.,П+отім бул+о муз+ичне уч+илище п+о кл+асу уд+арних інструм+ентів.,П+отім бул+о муз+ичне уч+илище п+о кл+асу уд+арних інструм+ентів.
2,але вона може займатися.,ал+е вон+а м+оже займ+атися.,ал+е вон+а м+оже займ+атися.
3,"Цю операцію потрібно провести якомога швидше. На цьому наголошують як українська влада, так і місцеві мешканці, яким вдалося виїхати з Маріуполя.","Ц+ю опер+ацію потр+ібно провест+и яком+ога шв+идше. Н+а цьом+у нагол+ошують +як укра+їнська вл+ада, т+ак +і місц+еві м+ешканці, як+им вдал+ося в+иїхати з Марі+уполя.","Ц+ю опер+ацію потр+ібно провест+и яком+ога шв+идше. Н+а цьом+у нагол+ошують +як укра+їнська вл+ада, т+ак +і місц+еві м+ешканці, як+им вдал+ося в+иїхати з Марі+уполя."
4,Тим часом Литва наказала своїм урядовцям позбутися телефонів к Сайомі та Хуавей.,Т+им ч+асом Литв+а наказ+ала св+оїм уряд+овцям позб+утися телеф+онів к Сай+омі т+а Хуав+ей.,Т+им ч+асом Литв+а наказ+ала св+оїм уряд+овцям позб+утися телеф+онів к Сай+омі т+а Хуав+ей.
5,пів години обговорював перспективи вступу україни в нато з президентом сша джо Байденом,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Байд+еном,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Б+айденом
6,"І також традиційно виголосив промову, звернувся.","+І т+акож традиц+ійно в+иголосив пром+ову, зверн+увся.","+І т+акож традиц+ійно в+иголосив пром+ову, зверн+увся."
7,"пояснює, що це все однозначно спричинене людською діяльністю.","по+яснює, щ+о ц+е вс+е однозн+ачно сприч+инене л+юдською ді+яльністю.","по+яснює, щ+о ц+е вс+е однозн+ачно сприч+инене л+юдською ді+яльністю."
8,"І Джо Байден пообіцяв, що всі, хто хоче виїхати, виїдуть.","+І Дж+о Байд+ен пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть.","+І Дж+о Б+айден пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть."
9,"Юрій Мамон, Євгенія Дюло, волос Америки, Каліфорнія.","+Юрій Мам+он, Євг+енія Д+юло, в+олос Ам+ерики, Каліф+орнія.","+Юрій Мам+он, Євг+енія Дюл+о, в+олос Ам+ерики, Каліф+орнія."


## Examples of inference on eval set (only with errors)

In [21]:
# only mismatched
df_errors_only = df_pred_highlighted[mismatch_mask]
display(HTML(df_errors_only.head(50).to_html(escape=False)))

,Input Text,Predictions,True Labels
5,пів години обговорював перспективи вступу україни в нато з президентом сша джо Байденом,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Байд+еном,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Б+айденом
8,"І Джо Байден пообіцяв, що всі, хто хоче виїхати, виїдуть.","+І Дж+о Байд+ен пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть.","+І Дж+о Б+айден пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть."
9,"Юрій Мамон, Євгенія Дюло, волос Америки, Каліфорнія.","+Юрій Мам+он, Євг+енія Д+юло, в+олос Ам+ерики, Каліф+орнія.","+Юрій Мам+он, Євг+енія Дюл+о, в+олос Ам+ерики, Каліф+орнія."
10,"Та й в цілому, уся долина Ріогранде вважається найбільш проблемним місцем на американсько-мексиканському кордоні.","Т+а й в ц+ілому, +уся дол+ина Ріогр+анде вваж+ається найб+ільш пробл+емним м+ісцем н+а америк+ансько-мексик+анському корд+оні.","Т+а й в ц+ілому, ус+я дол+ина Ріогр+анде вваж+ається найб+ільш пробл+емним м+ісцем н+а америк+ансько-мексик+анському корд+оні."
15,"Творці фонду розповідають, як майже тридцять років тому.","Тв+орці ф+онду розповід+ають, +як м+айже тр+идцять р+оків т+ому.","Творц+і ф+онду розповід+ають, +як м+айже тр+идцять р+оків том+у."
17,використання наданої фінансової допомоги військ процесами антикорупційних реформ в Україні.,в+икористання н+аданої фін+ансової допом+оги в+ійськ проц+есами антикорупц+ійних реф+орм в Укра+їні.,використ+ання н+аданої фін+ансової допом+оги в+ійськ проц+есами антикорупц+ійних реф+орм в Укра+їні.
19,"Інший кореспондент, він був серйозно пораний, йому ампутували ногу через обстріли російських військ.","+Інший кореспонд+ент, в+ін б+ув серй+озно п+ораний, й+ому ампутув+али н+огу ч+ерез +обстріли рос+ійських в+ійськ.","+Інший кореспонд+ент, в+ін б+ув серй+озно пор+аний, й+ому ампутув+али н+огу ч+ерез +обстріли рос+ійських в+ійськ."
29,аби стримати Росію від агресії.,аб+и стр+имати Рос+ію в+ід агр+есії.,аб+и стр+имати Р+осію в+ід агр+есії.
31,вони кажуть що відводять війська побачимо чи справді вони це зроблять побачимо чи відновлять вони це у найближчі місяці одне я можу сказати вам з певненістю як співголова українського кокусу та провідний член комітету закордонних справ який покриває європу,вон+и к+ажуть щ+о відв+одять військ+а поб+ачимо ч+и спр+авді вон+и ц+е зр+облять поб+ачимо ч+и відн+овлять вон+и ц+е +у найбл+ижчі місяц+і одн+е +я м+ожу сказ+ати в+ам з п+евненістю +як співголов+а укра+їнського к+окусу т+а пров+ідний чл+ен коміт+ету закорд+онних спр+ав як+ий покрив+ає євр+опу,вон+и к+ажуть щ+о відв+одять військ+а поб+ачимо ч+и спр+авді вон+и ц+е зр+облять поб+ачимо ч+и відн+овлять вон+и ц+е +у найбл+ижчі місяц+і одн+е +я м+ожу сказ+ати в+ам з п+евненістю +як співголов+а укра+їнського к+окусу т+а провідн+ий чл+ен коміт+ету закорд+онних спр+ав як+ий покрив+ає євр+опу
43,"то це, можливо, на його думку, був те, знати як.","т+о ц+е, можл+иво, н+а йог+о д+умку, б+ув т+е, зн+ати +як.","т+о ц+е, можл+иво, н+а йог+о д+умку, б+ув т+е, знат+и +як."
